# ПИД алгоритам за превртено нишало

## Референци:
- [Труд за превртено нишало](https://coneural.org/florian/papers/05_cart_pole.pdf)

In [1]:
import time
import threading

from IPython.lib.display import IFrame

import numpy as np
from plotly import graph_objects as go
from plotly import subplots
import ipywidgets as widgets

In [2]:
IFrame(src='https://drive.google.com/file/d/1QVOZr529GseqF8htFicXWtkc_r9HKLn-/preview', width='400', height='300')

In [2]:
class PID:
    def __init__(self, kp, ki=0, kd=0, dt=0.02):
        self.kp, self.ki, self.kd = kp, ki, kd
        self.kx = 0.002
        self.dt = dt
        self.max_actuator_output = 35
        self.reset()
    
    def reset(self):
        self.error_integral = 0
        self.previous_error = None
        self.previous_measurement = None
        
    def basic(self, setpoint, measurement):
        error = setpoint - measurement
        self.proportional_part = self.kp * error
        self.error_integral += error * self.dt
        self.integral_part = self.ki * self.error_integral
        if self.previous_error is None:
            self.previous_error = error
        self.derivative_part = self.kd * (error - self.previous_error) / self.dt
        self.output = self.proportional_part + self.integral_part + self.derivative_part
        self.previous_error = error
        return self.output
    
    def advanced(self, setpoint, measurement):
        
        x, x_dot, theta, theta_dot = measurement
        error = setpoint - theta + self.kx*( (8 - x) - x_dot) # greska na x  
        self.proportional_part = self.kp * error
        self.error_integral += self.ki * error * self.dt
        self.integral_part = np.clip(self.error_integral, 0, self.max_actuator_output)
        if self.previous_measurement is None:
            self.previous_measurement = theta
        self.derivative_part = self.kd * -(theta - self.previous_measurement) / self.dt
        self.output = self.proportional_part + self.integral_part + self.derivative_part
        self.previous_measurement = theta
        return self.output

    def get_output(self, setpoint, measurement):
        return self.advanced(setpoint, measurement)
    
    def get_controller_parameters(self, return_no_values=False):
        if return_no_values:
            return ['П', 'И', 'Д', 'ПИД']
        return {'П': self.proportional_part,
                'Д': self.derivative_part,
                'И': self.integral_part,
                'ПИД': self.output}
    
    def __str__(self):
        return f'ПИД управувач со параметри kp = {self.kp}, ki = {self.ki} и kd = {self.kd}.'

In [3]:
class BangBang:
    def __init__(self):
        self.max_actuator_output = 18
    
    def reset(self):
        pass
    
    def get_output(self, setpoint, measurement):
        self.output = 0 if measurement > setpoint else self.max_actuator_output
        return self.output
    
    def get_controller_parameters(self, return_no_values=False):
        if return_no_values:
            return ['Управувачки сигнал']
        return {'Управувачки сигнал': self.output}
    
    def __str__(self):
        return f'Двопозиционен управувач.'

In [4]:
class InvertedPendulumSimulation:
    def __init__(self, dt, initial_state, controller, fps=30):
        self.dt = dt
        self.dt_plot = 1 / fps
        self.render = True
        self.state = initial_state
        self.controller = controller
        self.controller_parameters = self.controller.get_controller_parameters(return_no_values=True)
        self.setpoint = (0,0)
        self.force = 0
        self.simstart = time.time()
        self.sim_ticks, self.max_sim_ticks = 0, 0
        self.history = []
        
        self.gravity = 9.81  # [m/s^2]
        self.cart_width = 0.5  # [m]
        self.pole_length = 2  # [m]
        self.cart_mass = 1  # [kg]
        self.pole_mass = 0.1  # [kg]
        self.max_force = 4* self.cart_mass * self.gravity  # [N]
        self.world_width, self.world_height = 16, 2  # [m]
        
        self.button_update_everything = widgets.Button(description='Застани')
        self.button_update_everything.on_click(self.clicked_button_update_everything)

        self.fig_sim = go.FigureWidget()
        self.fig_sim.add_scatter(mode='lines', name='Подлога')
        self.fig_sim.add_scatter(mode='lines', line_width=24, name='Количка')
        self.fig_sim.add_scatter(mode='lines+markers+text', line_width=3, marker_symbol='circle', marker_size=9, name='Сила')
        self.fig_sim.add_scatter(mode='lines', line_width=6, name='Прачка')
        self.fig_sim.update_xaxes(range=[0, self.world_width])
        self.fig_sim.update_yaxes(range=[-self.world_height/4, self.world_height], scaleanchor='x')
        self.fig_sim.update_layout(showlegend=False, transition_duration=0)
        self.fig_sim.update_layout(title='Изврши ја функцијата simulate() за да почнеш со симулацијата.')
        
        self.fig_data = go.FigureWidget(subplots.make_subplots(rows=2, cols=1, shared_xaxes=True))
        self.fig_data.add_scatter(x=[], y=[], mode='lines', name='Цел [m]')
        self.fig_data.add_scatter(x=[], y=[], mode='lines', name='Агол [m]')
        self.fig_data.add_scatter(x=[], y=[], mode='lines', name='Растoјание [m]')
        self.fig_data.add_scatter(x=[], y=[], mode='lines', name='Грешка [m]')
        self.fig_data.add_scatter(x=[], y=[], mode='lines', line_dash='dash', name='Заситување', row=2, col=1)
        for name in self.controller_parameters:
            self.fig_data.add_scatter(x=[], y=[], mode='lines', name=name, row=2, col=1)
        self.fig_data.update_yaxes(range=[-np.pi/2, np.pi/2], row=1, col=1)
        self.fig_data.update_layout(transition_duration=0)
        self.update_fig_sim()
        
        self.toggle_update = threading.Event()
        self.update_everything_thread = threading.Thread(name='painter', target=self.update_everything)
        self.update_everything_thread.start()
        
    def reset(self, initial_state):
        self.toggle_update.clear()
        self.state = initial_state
        self.update_fig_sim()
        self.sim_ticks, self.max_sim_ticks = 0, 0
        with self.fig_data.batch_update():
            for scatter in self.fig_data.data:
                scatter.x, scatter.y = [], []
        self.controller.reset()
    
    def show(self):
        self.box = widgets.VBox([self.button_update_everything, self.fig_sim, self.fig_data])
        return self.box
    
    def clicked_button_update_everything(self, button):
        if button.description == 'Продолжи':
            self.toggle_update.set()
            button.description = 'Застани'
        else:
            self.toggle_update.clear()
            button.description = 'Продолжи'

    def stop_sim(self, message=''):
        self.button_update_everything.disabled = True
        self.button_update_everything.description = 'Продолжи'
        self.toggle_update.clear()
        self.update_fig_data()
        self.fig_sim.update_layout(title=message)
    
    def simulate(self, setpoint, sim_time=10):
        if self.toggle_update.is_set():
            return 'Не брзај. Сѐ уште трае симулацијата. Повикај ја повторно оваа функција.'
        self.fig_sim.update_layout(title='Симулацијата е во тек.')
        self.setpoint = setpoint
        self.max_sim_ticks += int(sim_time / self.dt)
        self.fig_data.update_xaxes(range=[0, self.max_sim_ticks * self.dt])
        with self.fig_data.batch_update():
            now = self.sim_ticks * self.dt
            self.fig_data.data[0].x = self.fig_data.data[0].x + (now, now + sim_time)
            self.fig_data.data[0].y = self.fig_data.data[0].y + (self.setpoint,)*2
        self.button_update_everything.disabled = False
        self.button_update_everything.description = 'Застани'
        self.toggle_update.set()
    
    def sim_step(self, force):
        x, x_dot, theta, theta_dot = self.state
        l = self.pole_length / 2
        m = self.cart_mass + self.pole_mass
        sin_theta, cos_theta = np.sin(theta), np.cos(theta)
        temp = (force + self.pole_mass * l * theta_dot ** 2 * sin_theta) / m
        theta_ddot = (self.gravity * sin_theta - cos_theta * temp) / (l * (4/3 - self.pole_mass * cos_theta ** 2 / m))
        x_ddot = temp - self.pole_mass * theta_ddot * cos_theta / m
        state_dot = np.array([x_dot, x_ddot, theta_dot, theta_ddot])
        self.state = self.state + state_dot * self.dt
        def fix_angle(angle):
            angle %= 2*np.pi
            if angle > np.pi:
                angle -= 2*np.pi
            return angle
        self.state[2] = fix_angle(self.state[2])
    
    def actuator_model(self, force):
        force = -np.clip(force, -self.max_force, self.max_force)
        return force
    
    def update_everything(self):
        while True:
            time.sleep(self.dt_plot - (time.time() - self.simstart) % self.dt_plot)
            self.toggle_update.wait()
            for _ in range(round(self.dt_plot / self.dt)):
                self.update_sim()
            if self.render:
                self.update_fig_sim()
            if self.sim_ticks >= self.max_sim_ticks:
                self.stop_sim('Истече зададеното време за симулација.')
    
    def update_sim(self):
        x, x_dot, theta, theta_dot = self.state
        self.force = self.controller.get_output(self.setpoint, self.state)
        self.force = self.actuator_model(self.force)
        self.sim_step(self.force)

        x, x_dot, theta, theta_dot = self.state
        if not(0 < x < self.world_width) or not(-np.pi/2 < theta < np.pi/2):
            self.stop_sim('Количката избега надвор од теренот или пак прачката падна.')
        
        tick = self.sim_ticks * self.dt
        controller_parameters = self.controller.get_controller_parameters()
        self.history.append((self.state, self.setpoint, self.force, tick, controller_parameters))
        self.sim_ticks += 1

    def update_fig_sim(self):
        x, x_dot, theta, theta_dot = self.state
        fig = self.fig_sim
        with fig.batch_update():
            fig.data[0].x = [0, self.world_width]
            fig.data[0].y = [0, 0]
            fig.data[1].x = [x - self.cart_width / 2, x + self.cart_width / 2]
            fig.data[1].y = [0, 0]
            fig.data[2].x = [x, x + self.force / self.max_force * self.pole_length]
            fig.data[2].y = [-self.cart_width, -self.cart_width]
            arrow = 'triangle-right' if self.force > 0 else 'triangle-left' if self.force < 0 else 'circle'
            fig.data[2]['marker']['symbol'] = ['line-ns', arrow]
            textposition = 'right' if self.force == self.max_force else 'left' if self.force == -self.max_force else 'center'
            fig.data[2]['textposition'] = f'middle {textposition}'
            fig.data[2]['text'] = ['', 'MAX' if abs(self.force) == self.max_force else '' ]
            fig.data[3].x = [x, x + self.pole_length * np.sin(theta)]
            fig.data[3].y = [0, self.pole_length * np.cos(theta)]
            
    def update_fig_data(self):
        fig = self.fig_data
        history = self.history
        with fig.batch_update():
            ticks = tuple([data[3] for data in history])
            fig.data[1].x = fig.data[1].x + ticks
            fig.data[1].y = fig.data[1].y + tuple([data[0][2] for data in history])
            fig.data[2].x = fig.data[2].x + ticks
            fig.data[2].y = fig.data[2].y + tuple([(data[0][1])/10 for data in history])
            fig.data[3].x = fig.data[3].x + ticks
            fig.data[3].y = fig.data[3].y + tuple([(data[1] - data[0][2] + self.kx*( (8 - data[0][0]) + data[0][1]))/2 for data in history])
            fig.data[4].x = [0, self.sim_ticks * self.dt, None, 0, self.sim_ticks * self.dt]
            fig.data[4].y = [self.max_force, self.max_force, None, -self.max_force, -self.max_force]
            for index, name in enumerate(self.controller_parameters, start=5):
                fig.data[index].x = fig.data[index].x + ticks
                fig.data[index].y = fig.data[index].y + tuple([data[4][name] for data in history])
            self.history = []

In [5]:
pid = PID(kp=15, ki=0.0, kd=10)
initial_state = [3, 0, np.pi/20, 0]
sim = InvertedPendulumSimulation(dt=1/1000, initial_state=initial_state, controller=pid, fps=30)
sim.kx = 0.7 # zavisi od pocetnata brzina
sim.show()

    'data': [{'mode': 'lines…

In [6]:
sim.simulate(setpoint=0, sim_time=20)

In [8]:
sim.reset(initial_state)
sim.simulate(setpoint=0, sim_time=4)

## Задача:
- Нагодете го ПИД управувачот за успешно да се справува со различни почетни вредности на состојбениот вектор или различни конструкциски параметри (маса, должина).
- Искористете ја информацијата за позицијата по x-оската (првата вредност во состојбениот вектор) за количката да не избега надвор од теренот.

In [60]:
pid.kp, pid.ki,pid.kd = 15,0,1
sim.reset(initial_state)
sim.simulate(setpoint=0, sim_time=10)